In [10]:
import wandb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [57]:
# Initialize the API
api = wandb.Api()

# Specify your project details
entity = "haeri-hsn"  # Replace with your wandb entity
project = "stream_learning"  # Replace with your wandb project name

# Define filters
filters = {
    'state': 'finished',  # Only fetch finished runs
    'tags': {'$in': ['teconer_final_v3']},  # Runs containing a specific tag msmsa_horizon_analysis_melbourne_housing \ msmsa_anchor_analysis_melbourne_housing
    # 'CreatedAt': {'$gt': '2024-7-9'}  # Only fetch runs created after a specific date
}

# Query runs with filters
runs = api.runs(f"{entity}/{project}", filters=filters)

# print number of runs
print(f"Number of runs: {len(runs)}")

Number of runs: 24


In [61]:
# Extract data from each run
summary_list = []
config_list = []
name_list = []
for run in runs:
    # run.summary contains the output of the training
    summary_list.append(run.summary._json_dict)

    # run.config contains the hyperparameters
    config_list.append({k: v for k, v in run.config.items() if not k.startswith('_')})

    # run.name is the name of the run.
    name_list.append(run.name)

# Convert to DataFrame
summary_df = pd.DataFrame.from_records(summary_list)
config_df = pd.DataFrame.from_records(config_list)
name_df = pd.DataFrame({'name': name_list})

# Combine all parts into one DataFrame
df = pd.concat([name_df, config_df, summary_df], axis=1)
# remove the columns that are repeated
df['preview_window'] = df['preview_window'].astype('float')/60
df = df.loc[:, ~df.columns.duplicated()]
# change the name of the metrics and remove metric_ from the name
df = df.rename(columns=lambda x: x.replace("metric_", ""))
# Display the DataFrame
print(df.columns)

# print the values of the dataset (unique values)
print(df['dataset'].unique())




Index(['name', 'seed', 'tags', 'dataset', 'epsilon', 'wandb_log',
       'base_learner', 'online_model', 'average_training_size', '_wandb',
       'learning_model', 'average_records_per_trip', 'TMI', '_timestamp',
       '_step', 'MAE', 'preview_window', 'average_preview_records', '_runtime',
       'R2', 'RMSE', 'training_size_log'],
      dtype='object')
['teconer_helsinki_jan2018' 'teconer_helsinki_jan2018_100K']
[0.55 0.95 0.7 ]


## MAE, RMSE, R2 Barplots

In [62]:
%matplotlib qt
plt.close('all')


# datasets = list(df['dataset'].unique())
datasets = ['teconer_helsinki_jan2018']

for dataset in datasets:
    df_ = df[(df['dataset'] == dataset)]
    print(df_['epsilon'].unique())
    f, axs = plt.subplots(1, 3, figsize=(11, 4))
    for i, metric in enumerate(['R2', 'RMSE', 'MAE']):
        # plot barplot of R2 scores for different epsilon values and different preview durations
        sns.set_theme(style="whitegrid")
        # Try using .loc[row_indexer,col_indexer] = value instead
        sns.barplot(data=df_, x="epsilon", y=metric, hue="preview_window", errorbar="sd", palette="coolwarm", ax=axs[i])
        axs[i].set_xlabel("epsilon")
        axs[i].set_ylabel(metric)
        # rename legend title
        if i == 1:
            axs[i].legend(title="Preview [min]", ncol=df_['preview_window'].nunique(), loc='upper center', bbox_to_anchor=(0.5, 1.3))
        else:
            axs[i].get_legend().remove()
        
        print(df_.groupby(['epsilon', 'preview_window'])[metric].mean())
    # f.set_size_inches(12, 5)
    # f.suptitle(dataset)
    # increase the space between the plots
    plt.subplots_adjust(wspace=0.5, hspace=0.5, top=0.80, bottom=0.2)
    # plt.tight_layout()
    # save to \plots folder
    plt.savefig(f"teconer_metrics_{dataset}.svg")
    
plt.show()





[0.55 0.7 ]
epsilon  preview_window
0.55     1.0               0.777124
         5.0               0.611431
         10.0              0.518408
         30.0              0.353315
0.70     1.0               0.787812
         5.0               0.630611
         10.0              0.557675
         30.0              0.476934
Name: R2, dtype: float64
epsilon  preview_window
0.55     1.0               0.078991
         5.0               0.104299
         10.0              0.116096
         30.0              0.134540
0.70     1.0               0.077074
         5.0               0.101693
         10.0              0.111280
         30.0              0.121011
Name: RMSE, dtype: float64
epsilon  preview_window
0.55     1.0               0.042387
         5.0               0.060930
         10.0              0.070298
         30.0              0.083923
0.70     1.0               0.042134
         5.0               0.060078
         10.0              0.068295
         30.0              0.077628


In [59]:
%matplotlib qt
plt.close('all')


# datasets = list(df['dataset'].unique())
datasets = ['teconer_helsinki_jan2018']

for dataset in datasets:
    df_ = df[(df['dataset'] == dataset)]
    # f, axs = plt.subplots(1, 1, figsize=(5, 5))
    
    for i, metric in enumerate(['R2', 'RMSE', 'MAE']):
        plt.figure(dataset, figsize=(5, 5))
        # plot barplot of R2 scores for different epsilon values and different preview durations
        sns.set_theme(style="whitegrid")
        # Try using .loc[row_indexer,col_indexer] = value instead
        sns.barplot(data=df_, x="epsilon", y=metric, hue="preview_window", errorbar="sd", palette="coolwarm")
        plt.xlabel("epsilon")
        plt.ylabel(metric)
        # rename legend title
        # if i == 1:
        plt.legend(title="Preview [min]", ncol=df_['preview_window'].nunique(), loc='upper center', bbox_to_anchor=(0.5, 1.3))
        # else:
            # axs[i].get_legend().remove()
        
    # # f.set_size_inches(12, 5)
    # # f.suptitle(dataset)
    # # increase the space between the plots
    # plt.subplots_adjust(wspace=0.5, hspace=0.5, top=0.80, bottom=0.2)
        plt.tight_layout()
        # save to \plots folder
        plt.savefig(f"teconer_{dataset}_{metric}.svg")
    
plt.show()

In [19]:
%matplotlib qt


# print average_ahead_records for each dataset, and preview_duration in a table
df_ = df.groupby(['dataset', 'preview_window'])['average_preview_records'].mean().reset_index()
df_ = df_.pivot(index='dataset', columns='preview_window', values='average_preview_records')
df_ = df_.fillna(0)
# df_ = df_.sort_values(by='dataset', ascending=False)
# sort df_ based on the dataset names (given in the list)
# df_ = df_.reindex(['Teconer_Jan_10K', 'Teconer_Jan_100K', 'Teconer_Jan_1M','Teconer_Downtown_10K', 'Teconer_Downtown_100K', 'Teconer_Downtown_1M'])
df_ = df_.round(2)
sns.heatmap(df_, annot=True, cmap="coolwarm", fmt=".2f", cbar=False)
plt.title('Average number of queried preview points')
plt.xlabel('Preview [min]')
plt.ylabel('Dataset')
plt.tight_layout()
plt.show()


# print df 
print(df_)



preview_window                  1.0     5.0     10.0     30.0
dataset                                                      
teconer_helsinki_jan2018       52.04  244.44  458.18  1097.23
teconer_helsinki_jan2018_100K   3.77   14.26   26.18    60.28


In [147]:
# make a column 'Downtown' and 'Jan 2018' based on the dataset name
df['location'] = df['dataset'].apply(lambda x: 'Downtown' if 'Downtown' in x else 'Jan 2018')

# also make a column '10K', '100K', '1M' based on the dataset name
df['size'] = df['dataset'].apply(lambda x: '10K' if '10K' in x else ('100K' if '100K' in x else '1M'))

# plot a heatmap of average trip duration for different size values and location
df_ = df.groupby(['location', 'size'])['average_records_per_trip'].mean().reset_index()
df_ = df_.pivot(index='location', columns='size', values='average_records_per_trip')
df_ = df_.fillna(0)
df_ = df_.sort_values(by='location', ascending=False)
df_ = df_.round(2)
sns.heatmap(df_, annot=True, cmap="coolwarm", fmt=".2f", cbar=False)
plt.title('Average records per trip')
plt.xlabel('Number of Samples in Dataset')
plt.ylabel('')
plt.tight_layout()
plt.show()

# print the values in a table
print(df_)



size       100K    10K      1M
location                      
Jan 2018  95.88  11.25  956.94
Downtown  20.28   2.93  200.46
